In [1]:
import pandas as pd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
load_dotenv()
import os
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.alp_client_extractor import ALPClientExtractor
from database.adatabase import ADatabase

In [2]:
crypto_list = ["AAVE", "AVAX", "BAT", "BCH", 
 "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI", "UNI", "XTZ", "YFI"]
alp = ALPClientExtractor(os.getenv("APCAKEY"),os.getenv("APCASECRET"))

In [3]:
market = ADatabase("market")
market.connect()
index = market.retrieve("sp500")
market.disconnect()

In [4]:
start = datetime(2024,1,1)
end = datetime.now()

In [5]:
prices = []
date = start

for ticker in crypto_list:
    try:
        price = alp.crypto(ticker+"/USD",date,end)
        price.sort_values("date",inplace=True)
        price["date"] = pd.to_datetime(price["date"])
        date = date + timedelta(minutes=10000)
        prices.append(price)
    except Exception as e:
        print(str(e))

In [6]:
market_prices = pd.concat(prices)

In [15]:
def calculate_indicators(price, timeframe):
    """Calculate indicators for a single ticker."""
    price = price.sort_values("date")
    price["sma"] = price["adjclose"].rolling(timeframe).mean()
    price["ema"] = price["adjclose"].ewm(span=timeframe, adjust=False).mean()

    delta = price["adjclose"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(timeframe).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(timeframe).mean()
    rs = gain / loss
    price["rsi"] = 100 - (100 / (1 + rs))

    price["std"] = price["adjclose"].rolling(timeframe).std()
    price["bollinger_upper"] = price["sma"] + 2 * price["std"]
    price["bollinger_lower"] = price["sma"] - 2 * price["std"]

    price["pct_change"] = price["adjclose"].pct_change(periods=timeframe)
    price["momentum"] = price["adjclose"] - price["adjclose"].shift(timeframe)

    price["volatility"] = price["adjclose"].rolling(timeframe).std()
    price["coev"] = price["adjclose"].rolling(timeframe).std() / price["adjclose"].rolling(timeframe).mean()

    return price.dropna()

In [25]:
prices = []
timeframe = 5
for ticker in crypto_list:
    try:
        price = market_prices[market_prices["ticker"] == ticker + "/USD"]
        price = calculate_indicators(price, timeframe)
        prices.append(price)
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Combine and sort all price data
sim = pd.concat(prices).dropna().sort_values("date")
sim["year"] = sim["date"].dt.year
sim["week"] = [x.week for x in sim["date"]]
sim["sell_price"] = sim["adjclose"]

In [26]:
signals = ["sma"
           ,"ema","rsi","volatility","bollinger_upper","bollinger_lower","momentum","pct_change","coev"
          ]
ascendings = [True,False]

In [27]:
analysis = []
for signal in signals:
    for ascending in ascendings:
        opportunities = sim.groupby(["year","week","ticker"]).agg({"date":"last","adjclose":"first","sell_price":"last",signal:"first"}).reset_index()
        trades = opportunities.sort_values(signal,ascending=ascending).groupby(["year","week"]).first().reset_index()
        trades.sort_values("date",inplace=True)
        trades["return"] = trades["sell_price"] / trades["adjclose"]
        trades["pnl"] = trades["return"].cumprod()
        analysis.append({
            "signal":signal,
            "ascending":ascending,
            "pnl":trades["pnl"].iloc[-1]
        })

In [28]:
a = pd.DataFrame(analysis)

In [29]:
a.sort_values("pnl",ascending=False).head(50)

,signal,ascending,pnl
17,coev,False,4.323765
4,rsi,True,3.743138
14,pct_change,True,3.064256
13,momentum,False,2.742912
6,volatility,True,1.823460
7,volatility,False,1.784874
8,bollinger_upper,True,1.518457
10,bollinger_lower,True,1.518457
0,sma,True,1.518457
2,ema,True,1.518457
